In [33]:
import sys
sys.path.append("/home/rossidaniel/Documenti/bin-packing/Python/")
sys.path.append("/home/rossidaniel/Documenti/bin-packing/")

In [34]:
import Bin_Packing_Py as BP
import BP_print
import BP_plot
import BP_utilities
import HA_routines
import BP_instances

import matplotlib
%matplotlib nbagg
matplotlib.rcParams['figure.figsize'] = [6,3.5]

from ortools.linear_solver import pywraplp
import random
import time
import csv

import pandas as pd
from sqlalchemy import create_engine

In [35]:
class model_bin_packing:
    def __init__(self,item_list,bin_list):
        self.bp = BP.BinPack()
        self.bp.items.extend(item_list)
        self.bp.bins.extend(bin_list)
        
    def buildModel(self,amount_time=300000):
        solver = pywraplp.Solver('Model2DR', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
        #importo dimensioni dei pacchi e del camion
        n=len(self.bp.items)

        w =[self.bp.items[i].width for i in range(n)]
        d =[self.bp.items[i].depth for i in range(n)]
        h =[self.bp.items[i].height for i in range(n)]

        W =self.bp.bins[0].width
        D =solver.NumVar(0,sum([max(w[i],d[i]) for i in range(n)]),"D")
        H =self.bp.bins[0].height

        Mw=W+max(max(w),max(d))
        Md=self.bp.bins[0].depth+max(max(w),max(d))
        Mz=self.bp.bins[0].height+max(h)

        # definisco le variabili
        l =[[solver.BoolVar("l%d%d" % (i,j)) for i in range(n)] for j in range(n)]
        b =[[solver.BoolVar("b%d%d" % (i,j)) for i in range(n)] for j in range(n)]
        r =[solver.BoolVar("r%d" % (i)) for i in range(n)]

        x =[solver.NumVar(0,solver.infinity(),"x%d" % i) for i in range(n)]
        y =[solver.NumVar(0,solver.infinity(),"y%d" % i) for i in range(n)]


        #definisco i constraints
        for i in range(n):
            for j in range(n):
                if(i < j):
                    solver.Add(l[i][j] + l[j][i] + b[i][j] + b[j][i] >= 1)                           #(1)
                if(i != j):
                    solver.Add(x[i] - x[j] + Mw * l[i][j] <= Mw - w[i]*(1-r[i]) - d[i]*r[i])         #(2)
                    solver.Add(y[i] - y[j] + Md * b[i][j] <= Md - d[i]*(1-r[i]) - w[i]*r[i])         #(3)
            solver.Add(x[i] + w[i]*(1-r[i]) + d[i]*r[i] <= W)                                        #(4)
            solver.Add(y[i] + d[i]*(1-r[i]) + w[i]*r[i] <= D)                                        #(5)

        #funzione obiettivo
        objective = solver.Objective()
        objective.SetCoefficient(D,1)
        objective.SetMinimization()

        #soluzione
        solver.SetTimeLimit(amount_time)
        t=time.time()
        solver.Solve()
        t =time.time()-t

        s = BP.Solution(self.bp)
        for i in range(len(self.bp.items)):
            rt = False if r[i].solution_value() == 0 else True
            s.pushItem(self.bp.items[i],0,BP.Point(x[i].solution_value(),y[i].solution_value(),0),rt,False)
        return s,t,w,d,h,n
    
    def BuildSolution(self,s):
        print s
    
    def plotSolution(self,s):
        BP_plot.plotSolution(s)
    
    def plot3Dsolution(self,s):
        BP_plot.plot3DSolution(s,0)

    def feasible(self):
        return self.bp.bins[0].height*self.bp.bins[0].width >= sum([self.bp.items[i].width*self.bp.items[i].height for i in range(len(self.bp.items))])

In [28]:
n_items = []
Avg_w = []
Avg_d = []
Avg_a = []
Time = []

In [36]:
for i in range(100):
    items,bins=BP_instances.tc_sfida()
    model = model_bin_packing(items,bins)
    s,t,w,d,h,n=model.buildModel()
    #model.plotSolution(s)
    Time.append(round(t,3))
    Avg_w.append(round(sum(w)/n,3))
    Avg_d.append(round(sum(d)/n,3))
    Avg_a.append(round(sum([d[i]*w[i] for i in range(n)])/n,3))
    n_items.append(n)

In [37]:
dimensioni =pd.DataFrame({'n_items': n_items, 'Avg Width': Avg_w, 'Avg Depth' : Avg_d, 'Avg Area': Avg_a, 'Time': Time})

In [38]:
dimensioni.to_excel('result.xlsx', encoding='utf8')
dimensioni.to_latex('result.tex', encoding='utf8')
dimensioni.to_csv('result.csv', encoding='utf8')

In [39]:
dimensioni

,Avg Area,Avg Depth,Avg Width,Time,n_items
0,2.868,1.787,1.703,309.520,9
1,2.268,2.210,1.141,0.138,4
2,4.261,2.129,1.844,0.404,4
3,3.106,2.598,1.173,3.144,8
4,3.965,3.038,1.357,1.314,6
5,3.173,2.176,1.436,19.731,8
6,2.566,1.577,1.597,4.186,7
7,2.806,2.120,1.195,20.750,8
8,3.834,2.215,1.829,0.065,4
9,2.281,1.858,1.244,0.058,3
